# CatBoost v3.19 SMP Prediction Model Validation

**Model Performance:**
- MAPE: 5.28%
- R²: 0.827
- Features: 68

**Purpose:** RTM (Real-Time Market) single-step prediction

---

## 1. Setup & Installation

In [ ]:
# Install dependencies
!pip install catboost pandas numpy scikit-learn matplotlib seaborn -q
print("Dependencies installed!")

In [ ]:
# Clone repository
!git clone https://github.com/kiminbean/power-demand-forecast.git
%cd power-demand-forecast
print("Repository cloned!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from pathlib import Path
import json
from datetime import datetime, timedelta

print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print("Setup complete!")

## 2. Load Model & Metrics

In [ ]:
# Model paths
MODEL_DIR = Path("models/smp_v3_19_recent")
MODEL_PATH = MODEL_DIR / "catboost_model.cbm"
METRICS_PATH = MODEL_DIR / "metrics.json"

print(f"Model exists: {MODEL_PATH.exists()}")
print(f"Metrics exists: {METRICS_PATH.exists()}")

In [ ]:
# Load metrics
with open(METRICS_PATH) as f:
    metrics = json.load(f)

print("=" * 50)
print("CatBoost v3.19 Training Metrics")
print("=" * 50)
print(f"Test MAPE: {metrics['test_mape']:.2f}%")
print(f"Test R²: {metrics['test_r2']:.4f}")
print(f"Test MAE: {metrics['test_mae']:.2f}")
print(f"Validation R²: {metrics['val_r2']:.4f}")
print(f"Total samples: {metrics['n_samples']:,}")
print(f"Features: {metrics['n_features']}")
print(f"Best approach: {metrics['best_approach']}")
print("\nBest hyperparameters:")
for k, v in metrics['best_params'].items():
    print(f"  {k}: {v}")

In [ ]:
# Load CatBoost model
model = CatBoostRegressor()
model.load_model(str(MODEL_PATH))

print(f"Model loaded successfully!")
print(f"Feature count: {model.feature_count_}")

## 3. Load & Prepare Data

In [ ]:
# Load SMP data
DATA_PATH = Path("data/smp/smp_5years_epsis.csv")

df = pd.read_csv(DATA_PATH)
print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

In [ ]:
# Fix hour 24 issue and parse datetime
def fix_hour_24(ts):
    if ' 24:00' in str(ts):
        date_part = str(ts).replace(' 24:00', '')
        return pd.to_datetime(date_part) + pd.Timedelta(days=1)
    return pd.to_datetime(ts)

df['datetime'] = df['timestamp'].apply(fix_hour_24)
df = df[df['smp_mainland'] > 0].copy()
df = df.sort_values('datetime').reset_index(drop=True)

print(f"Clean data shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} ~ {df['datetime'].max()}")

## 4. Feature Engineering (68 Features)

In [ ]:
# Feature names (68 total)
FEATURE_NAMES = [
    # Time features (17)
    'hour', 'hour_sin', 'hour_cos',
    'dayofweek', 'dow_sin', 'dow_cos', 'is_weekend',
    'month', 'month_sin', 'month_cos',
    'doy_sin', 'doy_cos',
    'is_summer', 'is_winter',
    'peak_morning', 'peak_evening', 'off_peak',
    # Lag features (12)
    'smp_lag1', 'smp_lag2', 'smp_lag3', 'smp_lag4', 'smp_lag5', 'smp_lag6',
    'smp_lag12', 'smp_lag24', 'smp_lag48', 'smp_lag72', 'smp_lag96', 'smp_lag168',
    # Rolling statistics (24)
    'smp_ma6', 'smp_std6', 'smp_min6', 'smp_max6',
    'smp_ma12', 'smp_std12', 'smp_min12', 'smp_max12',
    'smp_ma24', 'smp_std24', 'smp_min24', 'smp_max24',
    'smp_ma48', 'smp_std48', 'smp_min48', 'smp_max48',
    'smp_ma72', 'smp_std72', 'smp_min72', 'smp_max72',
    'smp_ma168', 'smp_std168', 'smp_min168', 'smp_max168',
    # EMA features (5) - v3.19
    'smp_ema6', 'smp_ema12', 'smp_ema24', 'smp_ema48', 'smp_ema168',
    # Diff features (3)
    'smp_diff1', 'smp_diff24', 'smp_diff168',
    # Ratio features (2)
    'smp_lag1_vs_ma24', 'smp_lag1_vs_ma168',
    # Range features (2)
    'smp_range24', 'smp_range168',
    # Momentum features (3) - v3.19
    'smp_roc24', 'smp_roc168', 'smp_macd'
]

print(f"Total features: {len(FEATURE_NAMES)}")

In [ ]:
def create_features(df):
    """Create all 68 features for CatBoost v3.19"""
    data = df.copy()
    smp = data['smp_mainland']
    dt = data['datetime']
    
    # Time features
    data['hour'] = dt.dt.hour
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
    
    data['dayofweek'] = dt.dt.dayofweek
    data['dow_sin'] = np.sin(2 * np.pi * data['dayofweek'] / 7)
    data['dow_cos'] = np.cos(2 * np.pi * data['dayofweek'] / 7)
    data['is_weekend'] = (data['dayofweek'] >= 5).astype(float)
    
    data['month'] = dt.dt.month
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)
    
    doy = dt.dt.dayofyear
    data['doy_sin'] = np.sin(2 * np.pi * doy / 365)
    data['doy_cos'] = np.cos(2 * np.pi * doy / 365)
    
    data['is_summer'] = data['month'].isin([6, 7, 8]).astype(float)
    data['is_winter'] = data['month'].isin([12, 1, 2]).astype(float)
    
    data['peak_morning'] = ((data['hour'] >= 9) & (data['hour'] <= 12)).astype(float)
    data['peak_evening'] = ((data['hour'] >= 17) & (data['hour'] <= 21)).astype(float)
    data['off_peak'] = ((data['hour'] >= 1) & (data['hour'] <= 6)).astype(float)
    
    # Lag features
    for lag in [1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 96, 168]:
        data[f'smp_lag{lag}'] = smp.shift(lag)
    
    # Rolling statistics
    for window in [6, 12, 24, 48, 72, 168]:
        data[f'smp_ma{window}'] = smp.rolling(window).mean()
        data[f'smp_std{window}'] = smp.rolling(window).std()
        data[f'smp_min{window}'] = smp.rolling(window).min()
        data[f'smp_max{window}'] = smp.rolling(window).max()
    
    # EMA features (v3.19)
    for span in [6, 12, 24, 48, 168]:
        data[f'smp_ema{span}'] = smp.ewm(span=span, adjust=False).mean()
    
    # Diff features
    data['smp_diff1'] = smp.diff(1)
    data['smp_diff24'] = smp.diff(24)
    data['smp_diff168'] = smp.diff(168)
    
    # Ratio features
    data['smp_lag1_vs_ma24'] = data['smp_lag1'] / data['smp_ma24']
    data['smp_lag1_vs_ma168'] = data['smp_lag1'] / data['smp_ma168']
    
    # Range features
    data['smp_range24'] = data['smp_max24'] - data['smp_min24']
    data['smp_range168'] = data['smp_max168'] - data['smp_min168']
    
    # Momentum features (v3.19)
    data['smp_roc24'] = smp.pct_change(24)
    data['smp_roc168'] = smp.pct_change(168)
    data['smp_macd'] = data['smp_ema12'] - data['smp_ema24']
    
    return data

# Create features
df_features = create_features(df)
print(f"Features created: {df_features.shape}")

In [ ]:
# Prepare final dataset
df_clean = df_features.dropna().copy()

X = df_clean[FEATURE_NAMES].values
y = df_clean['smp_mainland'].values

print(f"Final dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Target range: {y.min():.2f} ~ {y.max():.2f}")

## 5. Model Validation

In [ ]:
# Split data (same as training: 80% train, 20% test)
split_idx = int(len(X) * 0.8)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Train: {len(X_train)} samples")
print(f"Test: {len(X_test)} samples")

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# Calculate metrics
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=" * 50)
print("Validation Results (Test Set)")
print("=" * 50)
print(f"MAPE: {mape:.2f}%")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")
print()
print("Comparison with Training Metrics:")
print(f"  MAPE: {mape:.2f}% vs {metrics['test_mape']:.2f}% (training)")
print(f"  R²: {r2:.4f} vs {metrics['test_r2']:.4f} (training)")

## 6. Visualization

In [ ]:
# Plot actual vs predicted
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Time series comparison (last 168 hours)
ax1 = axes[0, 0]
n_plot = 168  # 1 week
ax1.plot(y_test[-n_plot:], label='Actual', alpha=0.8)
ax1.plot(y_pred[-n_plot:], label='Predicted', alpha=0.8)
ax1.set_title('Last 168 Hours (1 Week)')
ax1.set_xlabel('Hour')
ax1.set_ylabel('SMP (won/kWh)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Scatter plot
ax2 = axes[0, 1]
ax2.scatter(y_test, y_pred, alpha=0.3, s=10)
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax2.set_title(f'Actual vs Predicted (R² = {r2:.4f})')
ax2.set_xlabel('Actual SMP')
ax2.set_ylabel('Predicted SMP')
ax2.grid(True, alpha=0.3)

# 3. Error distribution
ax3 = axes[1, 0]
errors = y_test - y_pred
ax3.hist(errors, bins=50, edgecolor='black', alpha=0.7)
ax3.axvline(0, color='r', linestyle='--', lw=2)
ax3.set_title(f'Error Distribution (MAE = {mae:.2f})')
ax3.set_xlabel('Error (won/kWh)')
ax3.set_ylabel('Frequency')
ax3.grid(True, alpha=0.3)

# 4. Percentage error distribution
ax4 = axes[1, 1]
pct_errors = (y_test - y_pred) / y_test * 100
ax4.hist(pct_errors, bins=50, edgecolor='black', alpha=0.7)
ax4.axvline(0, color='r', linestyle='--', lw=2)
ax4.set_title(f'Percentage Error Distribution (MAPE = {mape:.2f}%)')
ax4.set_xlabel('Error (%)')
ax4.set_ylabel('Frequency')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('catboost_v319_validation.png', dpi=150)
plt.show()

In [ ]:
# Feature importance
feature_importance = model.get_feature_importance()
importance_df = pd.DataFrame({
    'feature': FEATURE_NAMES,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(10, 8))
top_n = 20
plt.barh(range(top_n), importance_df['importance'].head(top_n).values[::-1])
plt.yticks(range(top_n), importance_df['feature'].head(top_n).values[::-1])
plt.xlabel('Importance')
plt.title(f'Top {top_n} Feature Importance')
plt.tight_layout()
plt.savefig('catboost_v319_feature_importance.png', dpi=150)
plt.show()

print("\nTop 10 Features:")
print(importance_df.head(10).to_string(index=False))

## 7. Hourly Performance Analysis

In [ ]:
# Analyze performance by hour
test_df = df_clean.iloc[split_idx:].copy()
test_df['predicted'] = y_pred
test_df['error'] = test_df['smp_mainland'] - test_df['predicted']
test_df['abs_pct_error'] = np.abs(test_df['error'] / test_df['smp_mainland']) * 100

hourly_mape = test_df.groupby('hour')['abs_pct_error'].mean()

plt.figure(figsize=(12, 5))
plt.bar(hourly_mape.index, hourly_mape.values, edgecolor='black', alpha=0.7)
plt.axhline(mape, color='r', linestyle='--', label=f'Overall MAPE: {mape:.2f}%')
plt.xlabel('Hour')
plt.ylabel('MAPE (%)')
plt.title('MAPE by Hour of Day')
plt.xticks(range(24))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('catboost_v319_hourly_mape.png', dpi=150)
plt.show()

print("\nHourly MAPE:")
print(hourly_mape.round(2))

## 8. Summary

In [ ]:
print("=" * 60)
print("CatBoost v3.19 SMP Prediction Model - Validation Summary")
print("=" * 60)
print()
print("Model Configuration:")
print(f"  - Features: {len(FEATURE_NAMES)}")
print(f"  - Best approach: {metrics['best_approach']}")
print(f"  - Training samples: {metrics['n_samples']:,}")
print()
print("Training Metrics:")
print(f"  - MAPE: {metrics['test_mape']:.2f}%")
print(f"  - R²: {metrics['test_r2']:.4f}")
print(f"  - MAE: {metrics['test_mae']:.2f}")
print()
print("Validation Metrics (Reproduced):")
print(f"  - MAPE: {mape:.2f}%")
print(f"  - R²: {r2:.4f}")
print(f"  - MAE: {mae:.2f}")
print()
print("Comparison with BiLSTM v3.2 (DAM model):")
print(f"  - CatBoost v3.19: MAPE {mape:.2f}%, R² {r2:.4f}")
print(f"  - BiLSTM v3.2:    MAPE 7.17%, R² 0.77")
print(f"  - Improvement:    {7.17 - mape:.2f}%p MAPE, {r2 - 0.77:.4f} R²")
print()
print("Purpose:")
print("  - RTM (Real-Time Market): CatBoost v3.19 (single-step)")
print("  - DAM (Day-Ahead Market): BiLSTM v3.2 (24-hour sequence)")
print("=" * 60)